In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import sklearn
import sys
import tensorflow as tf
import time

from tensorflow import keras
import tensorflow as tf
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

print(tf.__version__)
print(sys.version_info)
for module in np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.1.0
sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0)
numpy 1.18.1
pandas 1.0.1
sklearn 0.22.1
tensorflow 2.1.0
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf


In [2]:
train_dir = "training/training/"
valid_dir = "validation/validation/"
label_file = "monkey_labels.txt"

In [3]:
labels = pd.read_csv(label_file, header=0)
print(labels[' Common Name                   '])


0     mantled_howler                
1     patas_monkey                  
2     bald_uakari                   
3     japanese_macaque              
4     pygmy_marmoset                
5     white_headed_capuchin         
6     silvery_marmoset              
7     common_squirrel_monkey        
8     black_headed_night_monkey     
9     nilgiri_langur                
Name:  Common Name                   , dtype: object


In [11]:
height = 128
width = 128

channels = 3
batch_size = 16
num_classes = 10

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    #像素值 都除以255
    rescale = 1./255,
    # 图片随机旋转 (40度以内)
    rotation_range = 40,
    # 图片左右位移  20%限度以内
    width_shift_range = 0.2,
    # 图片上下位移  20%限度以内
    height_shift_range = 0.2,
    # 图像剪切强度
    shear_range = 0.2,
    # 图像缩放强度
    zoom_range = 0.2,
    # 是否水平翻转
    horizontal_flip = True,
    # 放大缩小吼， 像素填充方式
    fill_mode = 'nearest',
)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                  target_size = (height, width),
                                                  batch_size = batch_size,
                                                  seed = 666,
                                                  shuffle = True,
                                                  class_mode = "categorical")

valid_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
)
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                  target_size = (height, width),
                                                  batch_size = batch_size,
                                                  seed = 666,
                                                  shuffle = True,
                                                  class_mode = "categorical")

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [5]:
train_num = train_generator.samples
valid_num = valid_generator.samples

In [6]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size = 3, padding='same',
                       activation = 'selu', input_shape = [width, height, channels]),
    keras.layers.Conv2D(filters=32, kernel_size = 3, 
                        padding='same', activation = 'selu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=64, kernel_size = 3, 
                        padding='same', activation = 'selu'),
    keras.layers.Conv2D(filters=64, kernel_size = 3, 
                        padding='same', activation = 'selu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=128, kernel_size = 3, 
                        padding='same', activation = 'selu'),
    keras.layers.Conv2D(filters=128, kernel_size = 3, 
                        padding='same', activation = 'selu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation = 'selu'),
    
    keras.layers.Dense(num_classes, activation = 'softmax')
])

model.compile(loss="categorical_crossentropy",
             optimizer = "adam", metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 128)       7

In [10]:
tf.keras.backend.clear_session()

In [8]:
epochs = 10

history = model.fit_generator(train_generator, steps_per_epoch=train_num // batch_size,
                             epochs=epochs, validation_data=valid_generator,
                             validation_steps=valid_num//batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 68 steps, validate for 17 steps
Epoch 1/10
68/68 [==============================] - 25s 368ms/step - loss: 6.7546 - accuracy: 0.1165 - val_loss: 2.2209 - val_accuracy: 0.1397
Epoch 2/10
68/68 [==============================] - 25s 361ms/step - loss: 2.2687 - accuracy: 0.1516 - val_loss: 2.3563 - val_accuracy: 0.0993
Epoch 3/10
68/68 [==============================] - 25s 363ms/step - loss: 2.3302 - accuracy: 0.1007 - val_loss: 2.3150 - val_accuracy: 0.1176
Epoch 4/10
68/68 [==============================] - 25s 361ms/step - loss: 2.3215 - accuracy: 0.1081 - val_loss: 2.3148 - val_accuracy: 0.1103
Epoch 5/10
68/68 [==============================] - 25s 366ms/step - loss: 2.3248 - accuracy: 0.0860 - val_loss: 2.3276 - val_accuracy: 0.0956
Epoch 6/10
68/68 [==============================] - 25s 369ms/step - loss: 2.3303 - accuracy: 0.0933 - val_loss: 2.32

In [18]:
import tensorflow as tf
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import os


import tensorflow as tf


gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

# batch, classes, epochs
batch_size = 32
num_classes = 10
epochs = 50

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

# convert to float, normalise the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


# train 
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

Using TensorFlow backend.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs, 1 Logical GPU
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 16s 313us/step - loss: 2.0888 - accuracy: 0.2234 - val_loss: 1.8635 - val_accuracy: 0.3416
Epoch 2/50
50000/50000 [==============================] - 14s 273us/step - loss: 1.7507 - accuracy: 0.3593 - val_loss: 1.5682 - val_accuracy: 0.4260
Epoch 3/50
50000/50000 [==============================] - 12s 245us/step - loss: 1.5878 - accuracy: 0.4241 - val_loss: 1.5122 - val_accuracy: 0.4512
Epoch 4/50
50000/50000 [==============================] - 12s 247us/step - loss: 1.4827 - accuracy: 0.4625 - val_loss: 1.3406 - val_accuracy: 0.5168
Epoch 5/50
50000/50000 [==============================] - 13s 268us/step - loss: 1.4044 - accuracy: 0.4929 - val_loss: 1.2821 - val_accuracy: 0.5381
Epoch 6/50
50000/50000 [==========